根據把首頁往下滑，發現網站會對server送一個GET請求下一個30篇文章
- 2018(G) https://ithelp.ithome.com.tw/articles/10203216
- 2019(G) https://tlyu0419.github.io/2019/04/06/Crawl-Dcard/
- 2020 https://blog.jiatool.com/posts/dcard_api_v2/

### API root

- 看板資訊 http://dcard.tw/_api/forums
- 文章資訊 http://dcard.tw/_api/forums/{看板名稱}/posts
- 文章內文	http://dcard.tw/_api/posts/{文章編號}
- 文章內引用連結	http://dcard.tw/_api/posts/{文章編號}/links
- 文章內留言	http://dcard.tw/_api/posts/{文章編號}/comments

### API wirte
- 文章數量 https://www.dcard.tw/_api/posts?limit=100
- 文章分頁 http://dcard.tw/_api/posts/?before=文章id (before 回傳由新到舊 after舊到新)
- 文章留言 http://dcard.tw/_api/posts/文章id/comments
- 留言分頁 http://dcard.tw/_api/posts/文章id/comments?after=floor數字
- 熱門文章 https://dcard.tw/_api/posts?popular=true

### others
文章資訊(meta) 與 文章內留言 預設使用熱門度 (popularity) 作為排序，而且一次請求 (request) 中只回應 30 筆。
* 想要讓這兩項使用 時間 排序，可在 GET 參數中加入 popular=false
* 欲取得更多的 文章資訊(meta)，可以使用 before={某文章編號}來獲得早於 #{某文章編號} 的另外 30 筆 資訊。
* 欲取得更多的 文章內留言，可以使用 after={某樓層} 來獲得大於 #{某樓層} 的另外 30 筆 留言。

In [3]:
import requests
import json
import pandas as pd    # 引用套件並縮寫為 pd
from bs4 import BeautifulSoup
# https://www.dcard.tw/f/makeup

GET 美妝版首頁，Use BeautifulSoup with html, wrtie a txt

 

In [10]:
p = requests.Session()                             # 把這一次的request都算在同一個session裡
url=requests.get("https://www.dcard.tw/f/makeup")  # 請求的網址
soup = BeautifulSoup(url.text,"html.parser")
sel = soup.select("h2.tgn9uw-2 a")
sel

#for link in sel:
#    a.append(link["href"])
    #print('https://www.dcard.tw'+link.get('href'))

[]

In [5]:
# slope the title
def Catch(num):
    try:
        url = 'https://www.dcard.tw/_api/posts/'+str(num)  #api post with ID
        resq = requests.get(url)
        rejs = resq.json()
        return(pd.DataFrame(
            data=
            [{'id':rejs['id'],
              'title':rejs['title'],
              'content':rejs['content'],
              'excerpt':rejs['excerpt'],
              'createdAt':rejs['createdAt'],
              'updatedAt': rejs['updatedAt' ],
              'forumName':rejs['forumName'],
              'forumAlias':rejs['forumAlias'],
              'gender':rejs['gender'],
              'topics':rejs['topics']}],
            columns=['id', 'title', 'content', 'excerpt', 'createdAt', 
                     'updatedAt', 'forumName', 'forumAlias','gender', 'topics '])
              )
    except:
        print('data error...',i,'J:',j)
        return(pd.DataFrame(
            data=
            [{'id':"",
              'title':"",
              'content':"",
              'excerpt':"",
              'createdAt':"",
              'updatedAt': "",
              'forumName':"",
              'forumAlias':"",
              'gender':"",
              'topics':""}],
            columns=['id', 'title', 'content', 'excerpt', 'createdAt', 
                     'updatedAt', 'forumName', 'forumAlias','gender', 'topics '])
              )

In [6]:
# KeyError: 'id'
# Oh！文章不見了 這篇文章已被作者刪除。 按這裡 看更多文章吧
# https://www.dcard.tw/f/makeup/p/235079399
# Catch(235079399)
# url = "https://www.dcard.tw/_api/forums/makeup/posts?popular=false&limit=100&before=233254532" 

In [7]:
#url = "http://dcard.tw/_api/forums/makeup/posts?popular=false&limit=1"
url = "https://www.dcard.tw/_api/forums/makeup/posts?popular=false&limit=100&before=229872382" 
resq = requests.get(url).json()
resq

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [2]:
import random
import time

#url = "http://dcard.tw/_api/forums/makeup/posts?popular=false&limit=1"
url = "https://www.dcard.tw/_api/forums/makeup/posts?popular=false&limit=100&before=229872382" 
resq = requests.get(url)
rejs = resq.json()
df = pd.DataFrame()
df = df.append(Catch(rejs[0]['id']),ignore_index=True) 
#df

NameError: name 'requests' is not defined

In [ ]:
count = 0
delay_choices = [13,25,28,32]  # delay second
delay = random.choice(delay_choices) 

for j in range(300):
    last = str(int(df['id'].tail(1))) # 找出爬出資料的最後一筆ID
    print(last)
    url = "https://www.dcard.tw/_api/forums/makeup/posts?popular=false&limit=100&before=" + last
    resq = requests.get(url)
    rejs = resq.json()
    delay = random.choice(delay_choices)
    time.sleep(delay)  
    for i in range(len(rejs)):
        df = df.append(Catch(rejs[i]['id']), ignore_index=True)
        count = count+1
        print('data reading...',count,'id:',rejs[i]['id'],rejs[i]['title'])
    df.to_csv(str(j) + '_Dcard.csv', index=False)
    
df.to_csv('data/Dcard_makeup.csv', index=False) #encoding = "UTF-8"
print(df.shape)
df


In [ ]:
import random
import time

count = 0
delay_choices = [3,7,5]  # delay second
delay = random.choice(delay_choices) 

for j in range(5):
    print('data reading...',j,delay)
    delay = random.choice(delay_choices)
    time.sleep(delay)  

# ---------------正式版本分隔線---------------
- 測試版本代號：1.2  2020
- 正式版本代號：2.03  2021
- 開發人員： EGG Jerry Three


dva.x_x 版本開發

In [ ]:
#找到 dav.x_x 發表過的文章

p = requests.Session()                             # 把這一次的request都算在同一個session裡
url=requests.get("https://www.dcard.tw/@dva.x_x")  # 請求的網址
soup = BeautifulSoup(url.text,"html.parser")
sel = soup.select("h2.tgn9uw-2 a")
sel
a=[]
for link in sel:
    a.append(link["href"])
    print(link.get('href')[-9:])
    print('https://www.dcard.tw'+link.get('href'))

In [ ]:
# slope the title
def CatchUser(num):
    try:
        url = 'https://www.dcard.tw/_api/posts/'+str(num)  #api post with ID
        resq = requests.get(url)
        rejs = resq.json()
        return(pd.DataFrame(
            data=
            [{'id':rejs['id'],
              'department':rejs['department'],  #for author
              'title':rejs['title'],
              'content':rejs['content'],
              'excerpt':rejs['excerpt'],
              'createdAt':rejs['createdAt'],
              'updatedAt': rejs['updatedAt' ],
              'forumName':rejs['forumName'],
              'forumAlias':rejs['forumAlias'],
              'gender':rejs['gender'],
              'topics':rejs['topics']}],
            columns=['id', 'department', 'title', 'content', 'excerpt', 'createdAt', 
                     'updatedAt', 'forumName', 'forumAlias','gender', 'topics '])
              )
    except:
        print('data error...',i,'J:',j)
        return(pd.DataFrame(
            data=
            [{'id':"",
              'department':"",    #for author
              'title':"",
              'content':"",
              'excerpt':"",
              'createdAt':"",
              'updatedAt': "",
              'forumName':"",
              'forumAlias':"",
              'gender':"",
              'topics':""}],
            columns=['id', 'department', 'title', 'content', 'excerpt', 'createdAt', 
                     'updatedAt', 'forumName', 'forumAlias','gender', 'topics '])
              )

In [ ]:
# 熱門美妝版測試 (100筆)
url = 'https://www.dcard.tw/@dva.x_x'
#url = 'http://dcard.tw/_api/forums/makeup/posts?popular=true&limit=5'
resq = requests.get(url)
rejs = resq.json()
df = pd.DataFrame()
for i in range(len(rejs)):
    df = df.append(CatchUser(rejs[i]['id']),ignore_index=True)   #0 - 29
print(df.shape)
#df.to_csv('data/Op_Dcard_markeup.csv', index=False)
df
# 99	235031315	#請益 #問 LA girl 遮瑕挑色
# 235066991

In [ ]:
import random
import time

url = "http://dcard.tw/_api/forums/makeup/posts?popular=false&limit=1"
resq = requests.get(url)
rejs = resq.json()
df = pd.DataFrame()
df = df.append(CatchUser(rejs[0]['id']),ignore_index=True) 
#df
print(str(int(df['id'].tail(1))) )

count = 0
delay_choices = [1, 3, 5]  # delay second
delay = random.choice(delay_choices) 

for j in range(1):
    last = str(int(df['id'].tail(1))) # 找出爬出資料的最後一筆ID
    print(last)
    url = "https://www.dcard.tw/_api/forums/makeup/posts?popular=false&limit=100&before=" + last
    resq = requests.get(url)
    rejs = resq.json()
    for i in range(len(rejs)):
        df = df.append(Catch(rejs[i]['id']), ignore_index=True)
        count = count+1
        print('data reading...',count,'id:',rejs[i]['id'],rejs[i]['title'])
    delay = random.choice(delay_choices)
    time.sleep(delay)  
df.to_csv('data/Op_Dcard_markeup.csv', index=False)
print(df.shape)
df

In [ ]:
# slope the title
def CatchPost(num):
    try:
        url = 'https://www.dcard.tw/_api/posts/'+str(num)  #api post with ID
        resq = requests.get(url)
        rejs = resq.json()
        return(pd.DataFrame(
            data=
            [{'id':rejs['id'],
              'title':rejs['title'],
              'content':rejs['content'],
              'excerpt':rejs['excerpt'],
              'createdAt':rejs['createdAt'],
              'updatedAt': rejs['updatedAt' ],
              'forumName':rejs['forumName'],
              'forumAlias':rejs['forumAlias'],
              'gender':rejs['gender'],
              'topics':rejs['topics']}],
            columns=['id', 'title', 'content', 'excerpt', 'createdAt', 
                     'updatedAt', 'forumName', 'forumAlias','gender', 'topics '])
              )
    except:
        print('data error...',i,'J:',j)
        return(pd.DataFrame(
            data=
            [{'id':"",
              'title':"",
              'content':"",
              'excerpt':"",
              'createdAt':"",
              'updatedAt': "",
              'forumName':"",
              'forumAlias':"",
              'gender':"",
              'topics':""}],
            columns=['id', 'title', 'content', 'excerpt', 'createdAt', 
                     'updatedAt', 'forumName', 'forumAlias','gender', 'topics '])
              )

In [ ]:
#CatchPost(234999065)
#url = 'https://www.dcard.tw/_api/posts/'+str(num)  #api post with ID
url = 'https://www.dcard.tw/_api/posts/235103419'
resq = requests.get(url)
rejs = resq.json()
rejs

In [ ]:
url = 'https://www.dcard.tw/_api/posts/234912821'
resq = requests.get(url)
rejs = resq.json()
rejs

In [ ]:
# url = 'https://www.dcard.tw/_api/posts/235103302'
resq = requests.get(url)
rejs = resq.json()
rejs

# --------------- dva.x_x 版本分隔線 ---------------
- 測試版本代號：Jerry 1.0
- 開發人員： Jerry


In [ ]:
#test = open("data/test.txt","w",encoding='UTF-8')

In [ ]:
# asb
import random
import time

url = "http://dcard.tw/_api/forums/makeup/posts?popular=true&limit=99"
resq = requests.get(url)
rejs = resq.json()
df = pd.DataFrame()
df = df.append(Catch(rejs[0]['id']),ignore_index=True)   #0 - 29

delay_choices = [1]#, 3, 5, 7,20]  #延遲的秒數
delay = random.choice(delay_choices)  #隨機選取秒數

for j in range(10):
    last = str(int(df['id'].tail(1))) # 找出爬出資料的最後一筆ID
    url = "https://www.dcard.tw/_api/forums/makeup/posts?popular=true&limit=99&before=" + last
    resq = requests.get(url)
    rejs = resq.json()
    for i in range(len(rejs)):
        df = df.append(Catch(rejs[i]['id']), ignore_index=True)
        if i % 101 == 0:
            print('第',j,'次執行，data reading...',i,'last',last,'chocice',delay)
    delay = random.choice(delay_choices)  #隨機選取秒數
    time.sleep(delay)  #延遲

df.to_csv('data/Op_Dcard_markeup.csv', index=False)
print(df.shape)
df

In [ ]:
# 熱門美妝版測試 (100筆)
url = 'http://dcard.tw/_api/forums/makeup/posts?popular=true&limit=100'
resq = requests.get(url)
rejs = resq.json()
df = pd.DataFrame()
for i in range(len(rejs)):
    df = df.append(Catch(rejs[i]['id']),ignore_index=True)   #0 - 29
    if i % 20 == 0:
        print('data reading...',i)
print(df.shape)
df.to_csv('data/Op_Dcard_markeup.csv', index=False)
df
# 99	235031315	#請益 #問 LA girl 遮瑕挑色
# 235066991

In [ ]:
#last = str(int(df['ID'].tail(1))) #last dt
#last

In [ ]:
# 透過迴圈讀取10*100篇文章，若需讀取更多資料，可以將range(10)中的數值提升
for j in range(10):
    last = str(int(df['ID'].tail(1))) # 找出爬出資料的最後一筆ID
    url = 'https://www.dcard.tw/_api/posts?popular=true&before=' + last
    #url = 'https://www.dcard.tw/_api/posts?popular=true&limit=100&before=' + last
    resq = requests.get(url)
    rejs = resq.json()
    for i in range(len(rejs)):
        df = df.append(Catch(rejs[i]['id']), ignore_index=True)
        if i % 10 == 0:
            print('第',j,'次執行，data reading...',i)
            
print(df.shape)
df

In [ ]:
last = str(int(df['ID'].tail(1)))
url = 'http://dcard.tw/_api/forums/makeup/posts?popular=true&limit=100&before=' + last
resq = requests.get(url)
rejs = resq.json()
df = pd.DataFrame()
df

In [ ]:
# 熱門美妝版測試 (100筆)

for i in range(10):
    last = str(int(df['ID'].tail(1)))
    url = 'http://dcard.tw/_api/forums/makeup/posts?popular=true&limit=100&before=' + last
    resq = requests.get(url)
    rejs = resq.json()
    df = pd.DataFrame()
    for i in range(len(rejs)):
        df = df.append(Catch(rejs[i]['id']),ignore_index=True)
        if i % 20 == 0:
            print('data reading...',i)
    print(df.shape)

df.to_csv('data/Op_Dcard_markeup.csv', index=False)
df

# 99	235031315	#請益 #問 LA girl 遮瑕挑色

In [ ]:
last = str(int(df.tail(1).ID)) # 找出爬出資料的最後一筆ID '235031315'
url = 'http://dcard.tw/_api/forums/makeup/posts?popular=true&limit=100&before=' + last
resq = requests.get(url)
rejs = resq.json()
for i in range(len(rejs)):
    df = df.append(Catch(rejs[i]['id']), ignore_index=True)
print(df.shape)


df

In [ ]:

#df

In [ ]:
##ok ^
for j in range(3):
    last = str(int(df.tail(1).ID)) # 找出爬出資料的最後一筆ID
    url = 'http://dcard.tw/_api/forums/makeup/posts?popular=true&limit=100&before=' + last
    resq = requests.get(url)
    rejs = resq.json()
    for i in range(len(rejs)):
        df = df.append(Catch(rejs[i]['id']), ignore_index=True)
print(df.shape)
#df

In [ ]:
# 透過迴圈讀取10*100篇文章，若需讀取更多資料，可以將range(10)中的數值提升
for j in range(10):
    last = str(int(df.tail(1).ID)) # 找出爬出資料的最後一筆ID
    url = 'http://dcard.tw/_api/forums/makeup/posts?popular=true&limit=100&before=' + last
    resq = requests.get(url)
    rejs = resq.json()
    for i in range(len(rejs)):
        df = df.append(Catch(rejs[i]['id']), ignore_index=True)
        if i % 20 == 0:
            print('data reading...',i)
print(df.shape)
df


In [ ]:
df.to_excel('data/Dcard_markeup.excel', index=False)

欄位	說明	備註
ID	編號	
title	標題	
content	內容	
excerpt	摘要	
createdAt	發布時間	
updatedAt	更新時間	
commentCount	留言數	
forumName	分類	中文
forumAlias	分類	英文
gender	性別	
likeCount	心情數量	
reactions	心情細項	把以上心情細分為「愛心」、「哈哈」、「跪」、「森77」、「驚訝」等類型
topics	標籤	

In [ ]:
# 固定碼測試
def Catch_test(ID):
    url = 'https://www.dcard.tw/_api/posts/'+str(ID)
    requ = requests.get(url)
    rejs = requ.json()
    #rejs
    return(pd.DataFrame(
        data=
        [{'ID':rejs['id'],
          'title':rejs['title'],
          'content':rejs['content'],
          'excerpt':rejs['excerpt'],
          'createdAt':rejs['createdAt'],
          'updatedAt': rejs['updatedAt' ],
          'forumName':rejs['forumName'],
          'forumAlias':rejs['forumAlias'],
          'gender':rejs['gender'],
          'topics':rejs['topics']}],
        columns=['ID', 'title', 'content', 'excerpt', 'createdAt', 
                 'updatedAt', 'forumName', 'forumAlias','gender', 'topics '])
          )
Catch_test(235020702)